In [1]:
from jupyter_plotly_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output

import requests
import json
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [2]:
def get_country_codes():
    country_codes_query = '''
    {
        summary {
            countries {
                Code
        }
      }
    }

    '''
    url = "https://api-corona.azurewebsites.net/graphql"
    country_codes_request = requests.post(url, json={"query": country_codes_query})
    if country_codes_request.status_code != 200:
        raise Exception("Query failed")
        
    country_codes = country_codes_request.json()['data']['summary']['countries']
    return [e['Code'] for e in country_codes if e['Code'] is not None]

print(get_country_codes())

['AF', 'AL', 'DZ', 'AD', 'AO', 'AG', 'AM', 'AU', 'AT', 'AZ', 'BS', 'BH', 'BD', 'BB', 'BY', 'BE', 'BZ', 'BJ', 'BT', 'BO', 'BA', 'BW', 'BN', 'BG', 'BF', 'BU', 'BI', 'CV', 'KH', 'CM', 'CA', 'CF', 'TD', 'CN', 'KM', 'CG', 'CD', 'CR', 'CI', 'HR', 'CU', 'CY', 'CZ', 'DK', 'ST', 'DJ', 'DM', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'EE', 'SZ', 'ET', 'FJ', 'FI', 'GA', 'GM', 'GE', 'GH', 'GR', 'GD', 'GT', 'GN', 'GW', 'GY', 'HT', 'holy-see', 'HN', 'HU', 'IS', 'ID', 'IR', 'IQ', 'IE', 'IL', 'JM', 'JP', 'JO', 'KZ', 'KE', 'KR', 'XK', 'KW', 'KG', 'LA', 'LV', 'LB', 'LS', 'LR', 'LY', 'LI', 'LT', 'LU', 'ms-zaandam', 'MG', 'MW', 'MY', 'MV', 'ML', 'MT', 'MR', 'MU', 'MD', 'MC', 'MN', 'ME', 'MA', 'MZ', 'NA', 'NP', 'NL', 'NZ', 'NI', 'NE', 'NG', 'MK', 'NO', 'OM', 'PK', 'PA', 'PG', 'PY', 'PE', 'PH', 'PL', 'PT', 'QA', 'RO', 'RU', 'RW', 'KN', 'LC', 'VC', 'SM', 'ST', 'SA', 'SN', 'RS', 'SC', 'SL', 'SG', 'SK', 'SI', 'SO', 'ZA', 'SS', 'ES', 'LK', 'SD', 'SR', 'SE', 'CH', 'SY', 'TW', 'TJ', 'TZ', 'TH', 'TL', 'TG', 'TT', 'TN', '

In [8]:
def get_country_name(country_code): 
    country_name_query = ''' 
    query getCountryName($countryCode: ID!) {
        country(country: $countryCode) {
            Summary {
                Slug
            }
        }
    }
    '''
    body = {
      "query": country_name_query,
      "variables": {"countryCode": country_code}
    }
    
    url = "https://api-corona.azurewebsites.net/graphql"
    country_name_request = requests.post(url, json=body)
    
    if country_name_request.status_code != 200:
        raise Exception("Query failed.")
    return country_name_request.json()['data']['country']['Summary']['Slug']

print(get_country_name("US"))

united-states


In [21]:
def get_cases_data(country_code):
    country_data_query = ''' 
    query getCountryData($countryCode: ID!) {
        timelineCountry(country: $countryCode) {
            Country
            Date
            Confirmed
            Deaths
            Lat
            Long
        }
    } 
    '''
    body = {
      "query": country_data_query,
      "variables": {"countryCode": country_code}
    }

    url = "https://api-corona.azurewebsites.net/graphql"
    country_data_request = requests.post(url, json=body)

    if country_data_request.status_code != 200:
        raise Exception("Query failed")
        
    country_info = country_data_request.json()['data']['timelineCountry']
    cases_info = {e['Date']: {"Confirmed": e['Confirmed'], "Deaths": e['Deaths']} for e in country_info} 
    return cases_info

print(get_cases_data("US"))

{'01-22-2020': {'Confirmed': 1, 'Deaths': 0}, '01-23-2020': {'Confirmed': 1, 'Deaths': 0}, '01-24-2020': {'Confirmed': 2, 'Deaths': 0}, '01-25-2020': {'Confirmed': 2, 'Deaths': 0}, '01-26-2020': {'Confirmed': 5, 'Deaths': 0}, '01-27-2020': {'Confirmed': 5, 'Deaths': 0}, '01-28-2020': {'Confirmed': 5, 'Deaths': 0}, '01-29-2020': {'Confirmed': 5, 'Deaths': 0}, '01-30-2020': {'Confirmed': 5, 'Deaths': 0}, '01-31-2020': {'Confirmed': 7, 'Deaths': 0}, '02-01-2020': {'Confirmed': 8, 'Deaths': 0}, '02-02-2020': {'Confirmed': 8, 'Deaths': 0}, '02-03-2020': {'Confirmed': 11, 'Deaths': 0}, '02-04-2020': {'Confirmed': 11, 'Deaths': 0}, '02-05-2020': {'Confirmed': 11, 'Deaths': 0}, '02-06-2020': {'Confirmed': 11, 'Deaths': 0}, '02-07-2020': {'Confirmed': 11, 'Deaths': 0}, '02-08-2020': {'Confirmed': 11, 'Deaths': 0}, '02-09-2020': {'Confirmed': 11, 'Deaths': 0}, '02-10-2020': {'Confirmed': 11, 'Deaths': 0}, '02-11-2020': {'Confirmed': 12, 'Deaths': 0}, '02-12-2020': {'Confirmed': 12, 'Deaths': 0},

In [25]:
def get_location_data(country_code):
    response = requests.get("https://corona.azure-api.net/country/" + country_code)
    country_summary = response.json()["Summary"]
    return {"Lat": country_summary["Lat"], "Long": country_summary["Long_"]}
print(get_location_data("US"))

{'Lat': 38, 'Long': -97}


In [28]:
country_codes = get_country_codes()
rows = []
for code in country_codes:
    country_name = get_country_name(code)
    cases_info = get_cases_data(code)
    location_info = get_location_data(code)
    for date in cases_info.keys():

        confirmed = cases_info[date]["Confirmed"]
        log10_confirmed = max(0, np.log10(confirmed))
        ln_confirmed = max(0, np.log(confirmed))

        deaths = cases_info[date]["Deaths"]
        log10_deaths = max(0, np.log10(deaths))
        ln_deaths = max(0, np.log(deaths))

        rows.append([code, country_name, location_info["Lat"], location_info["Long"], date, 
                     confirmed, deaths, log10_confirmed, log10_deaths, ln_confirmed, ln_deaths])

summary_df = pd.DataFrame(rows, columns=['Country Code', 'Country', 'Lat', 'Long', 'Date', 'Confirmed', "Deaths",
                                        "Log10 Confirmed", "Log10 Deaths", "Ln Confirmed", "Ln Deaths"])

In [29]:
summary_df.to_csv("./data/graphql.csv")

In [30]:
summary_df = pd.read_csv('./data/graphql.csv')
summary_df.head(10)

,Unnamed: 0,Country Code,Country,Lat,Long,Date,Confirmed,Deaths,Log10 Confirmed,Log10 Deaths,Ln Confirmed,Ln Deaths
0,0,AF,afghanistan,33.0,65.0,01-22-2020,0,0,0.0,0.0,0.0,0.0
1,1,AF,afghanistan,33.0,65.0,01-23-2020,0,0,0.0,0.0,0.0,0.0
2,2,AF,afghanistan,33.0,65.0,01-24-2020,0,0,0.0,0.0,0.0,0.0
3,3,AF,afghanistan,33.0,65.0,01-25-2020,0,0,0.0,0.0,0.0,0.0
4,4,AF,afghanistan,33.0,65.0,01-26-2020,0,0,0.0,0.0,0.0,0.0
5,5,AF,afghanistan,33.0,65.0,01-27-2020,0,0,0.0,0.0,0.0,0.0
6,6,AF,afghanistan,33.0,65.0,01-28-2020,0,0,0.0,0.0,0.0,0.0
7,7,AF,afghanistan,33.0,65.0,01-29-2020,0,0,0.0,0.0,0.0,0.0
8,8,AF,afghanistan,33.0,65.0,01-30-2020,0,0,0.0,0.0,0.0,0.0
9,9,AF,afghanistan,33.0,65.0,01-31-2020,0,0,0.0,0.0,0.0,0.0


Playground for live-update code below

In [316]:
location_dict = {}
for code in country_codes:
    location_info, _ = get_country_data(code)
    location_dict[code] = location_info
location_dict

{'US': {'Lat': '40.0', 'Long': '-100.0'},
 'IT': {'Lat': '41.87194', 'Long': '12.56738'},
 'BR': {'Lat': '-14.235', 'Long': '-51.9253'},
 'RU': {'Lat': '61.52401', 'Long': '105.31875600000001'},
 'MX': {'Lat': '23.6345', 'Long': '-102.5528'},
 'JP': {'Lat': '36.204824', 'Long': '138.252924'},
 'CA': {'Lat': None, 'Long': None},
 'CO': {'Lat': '4.5709', 'Long': '-74.2973'},
 'PE': {'Lat': '-9.19', 'Long': '-75.0152'},
 'ES': {'Lat': '40.463667', 'Long': '-3.7492199999999998'},
 'IN': {'Lat': '20.593684', 'Long': '78.96288'},
 'GB': {'Lat': None, 'Long': None},
 'CN': {'Lat': None, 'Long': None},
 'CL': {'Lat': '-35.6751', 'Long': '-71.543'},
 'NL': {'Lat': None, 'Long': None},
 'AU': {'Lat': None, 'Long': None},
 'PK': {'Lat': '30.3753', 'Long': '69.3451'},
 'DE': {'Lat': '51.165690999999995', 'Long': '10.451526'},
 'SE': {'Lat': '60.128161', 'Long': '18.643501'},
 'UA': {'Lat': '48.3794', 'Long': '31.1656'},
 'DK': {'Lat': None, 'Long': None},
 'FR': {'Lat': None, 'Long': None},
 'AF':

In [317]:
with open('graphql_country_locations.json', 'w') as fp:
    json.dump(location_dict, fp)

In [7]:
with open('graphql_country_locations.json', 'r') as fp:
    location_dict = json.load(fp)

In [332]:
def fetch_new_data():
    new_data_query = '''
    {
        summary {
            countries {
                Code
                Confirmed
                Deaths
                Last_Update
                Slug
            }
        }
    }
    '''
    
    url = "https://api-corona.azurewebsites.net/graphql"
    new_data_request = requests.post(url, json={"query": new_data_query,})
    
    if new_data_request.status_code != 200:
        raise Exception("Query failed.")
    return new_data_request.json()['data']['summary']['countries']

print(fetch_new_data())

[{'Code': 'US', 'Confirmed': 3576157, 'Deaths': 138358, 'Last_Update': '2020-07-17 06:00:06', 'Slug': 'united-states'}, {'Code': 'IT', 'Confirmed': 243736, 'Deaths': 35017, 'Last_Update': '2020-07-17 06:00:08', 'Slug': 'italy'}, {'Code': 'BR', 'Confirmed': 2012151, 'Deaths': 76688, 'Last_Update': '2020-07-17 06:00:08', 'Slug': 'brazil'}, {'Code': 'RU', 'Confirmed': 751612, 'Deaths': 11920, 'Last_Update': '2020-07-17 06:00:09', 'Slug': 'russia'}, {'Code': 'MX', 'Confirmed': 324041, 'Deaths': 37574, 'Last_Update': '2020-07-17 06:00:10', 'Slug': 'mexico'}, {'Code': 'JP', 'Confirmed': 23510, 'Deaths': 985, 'Last_Update': '2020-07-17 06:00:10', 'Slug': 'japan'}, {'Code': 'CA', 'Confirmed': 111144, 'Deaths': 8875, 'Last_Update': '2020-07-17 06:00:11', 'Slug': 'canada'}, {'Code': 'CO', 'Confirmed': 165169, 'Deaths': 6164, 'Last_Update': '2020-07-17 06:00:11', 'Slug': 'colombia'}, {'Code': 'PE', 'Confirmed': 341586, 'Deaths': 12615, 'Last_Update': '2020-07-17 06:00:12', 'Slug': 'peru'}, {'Code

In [336]:
def update_data(df):
    new_rows = []
    new_data = fetch_new_data()
    for country_data in new_data:
        country_code = country_data['Code']
        if not country_code:
            continue
        current_date = datetime.strptime(country_data['Last_Update'].split(" ")[0], '%Y-%m-%d')
        current_date_formatted = current_date.strftime("%m-%d-%Y")
        current_date_df = df[(df['Date'] == current_date_formatted) & (df['Country Code'] == country_code)]
        
        # add new row if data corresponding to the current country and date does not exist
        if current_date_df.empty:
            row = {}
            row['Country Code'] = country_code
            row['Country'] = country_data['Slug']
            row['Lat'] = location_dict[country_code]['Lat']
            row['Long'] = location_dict[country_code]['Long']
            row['Date'] = current_date_formatted
            row['Confirmed'] = country_data['Confirmed']
            row['Deaths'] = country_data['Deaths']
            row['Log10 Confirmed'] = max(0, np.log10(country_data['Confirmed']))
            row['Log10 Deaths'] = max(0, np.log10(country_data['Deaths']))
            row['Ln Confirmed'] = max(0, np.log(country_data['Confirmed']))
            row['Ln Deaths'] = max(0, np.log(country_data['Deaths']))
            
            df = df.append(row, ignore_index=True)
    return df

In [342]:
summary_df = update_data(summary_df)
summary_df.shape

/Users/liambai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log10
/Users/liambai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in log


(32744, 12)

In [221]:
summary_df

,Unnamed: 0,Country Code,Country,Lat,Long,Date,Confirmed,Deaths,Log10 Confirmed,Log10 Deaths,Ln Confirmed,Ln Deaths
0,0,US,united-states,40.000000,-100.000000,01-22-2020,1,0,0.000000,0.000000,0.000000,0.000000
1,1,US,united-states,40.000000,-100.000000,01-23-2020,1,0,0.000000,0.000000,0.000000,0.000000
2,2,US,united-states,40.000000,-100.000000,01-24-2020,2,0,0.301030,0.000000,0.693147,0.000000
3,3,US,united-states,40.000000,-100.000000,01-25-2020,2,0,0.301030,0.000000,0.693147,0.000000
4,4,US,united-states,40.000000,-100.000000,01-26-2020,5,0,0.698970,0.000000,1.609438,0.000000
5,5,US,united-states,40.000000,-100.000000,01-27-2020,5,0,0.698970,0.000000,1.609438,0.000000
6,6,US,united-states,40.000000,-100.000000,01-28-2020,5,0,0.698970,0.000000,1.609438,0.000000
7,7,US,united-states,40.000000,-100.000000,01-29-2020,5,0,0.698970,0.000000,1.609438,0.000000
8,8,US,united-states,40.000000,-100.000000,01-30-2020,5,0,0.698970,0.000000,1.609438,0.000000
9,9,US,united-states,40.000000,-100.000000,01-31-2020,7,0,0.845098,0.000000,1.945910,0.000000


In [ ]:
def createMap(df, selected_category):
    return px.scatter_geo(df,
        lat = df['Lat'],
        lon = df['Long'],
        color=df["Log " + selected_category],
        opacity=0.4,
        hover_name=df['Country'],
        size=df["Log " + selected_category],
        animation_frame=df['Date'].astype(str),
        projection="natural earth")

In [177]:
app = JupyterDash("Covid Dashboard")

app.layout = html.Div([
    html.H1(id='title', children='Covid-19 Tracker'),
    dcc.RadioItems(
        id='type-selector',
        options=[{'label': i, 'value': i} for i in ['Confirmed', 'Deaths']],
        value='Confirmed',
        style={'margin-top': '60px'}
    ),
    dcc.Graph(id='covid-map'),
    dcc.Interval(
        id='interval-component',
        interval=10*1000, # in milliseconds
        n_intervals=0
    )
], style={'text-align':'center'})

@app.callback(
    Output('covid-map', 'figure'),
    [Input('type-selector', 'value'),
    Input('interval-component', 'n_intervals')])
def update_map(selected_type, n):
    ctx = dash.callback_context
    print(ctx.triggered)
    #if ctx.triggered:
#         if ctx.triggered[0]['prop_id'].split('.')[0] == 'interval-component':
#             selected_type = "Deaths"
    return createMap(summary_df, selected_type)

app